In [ ]:
import pandas as pd

In [ ]:
schools = pd.read_json('https://stem-mentoring.firebaseio.com/school.json')
cities = pd.read_json('https://stem-mentoring.firebaseio.com/cities.json')
students = pd.read_json('https://stem-mentoring.firebaseio.com/students.json').T
confirm = pd.read_json('https://stem-mentoring.firebaseio.com/confirm.json').T

In [ ]:
students['school_id'] = students.school.map(lambda x:x['id'])
students['school_name'] = students.school.map(lambda x:x['label'])
students['city_id'] = students.school.map(lambda x:x['city'])

In [ ]:
students.columns

# The content below is real code that generates list of participants.

In [ ]:
students = pd.read_json('https://stem-mentoring.firebaseio.com/students.json').T
students['school_name'] = students.school.map(lambda x:x['value'])
register_list = students[['school_name', 'order', 'name', 'grade', 'phone', 'guardian_name', 'guardian_phone', 'interests', 'motivation', 'timestamp']].sort_values(['school_name', 'order'])
register_list.to_excel('명단.xlsx', header=['학교', '순번', '이름', '학년', '연락처', '보호자', '보호자 연락처', '관심 학과', '동기', '접수 시간'], index=False)